# Progetto di Computer Science

## Davide Meloni, Alberto Raimondi
### Febbraio 2018

In [2]:
# importazione delle librerie
import pandas as pd
import numpy as np
import json
import operator
import re
import math

In [3]:
#importo i CSV
df_movies=pd.read_csv("tmdb_5000_movies.csv")
df_credits=pd.read_csv("tmdb_5000_credits.csv")
df_credits["id"]=df_credits["movie_id"]
df=df_movies.merge(df_credits, on=['id','title'],how='left')

1 For each movie, compute the number of cast members

In [4]:
#conto il numero di film in base all'id
df["no. cast members"]=df.cast.apply(lambda x: len(x))

2 How many movies do not have a homepage?

In [5]:
#conto il numero di film senza homepage in base all'id
df[df["homepage"].isnull()]['id'].count()

3091

3 For each year, how many movies do not have a homepage?

In [6]:
#creo la colonna "year" dai primi quattro caratteri dell'attributo "release_date" e conto il numero di film senza homepage per ogni anno in base all'id
df["year"]=df["release_date"].astype(str).str[:4]
df[df["homepage"].isnull()].groupby("year").count()["id"]

year
1916      1
1925      1
1927      1
1929      2
1930      1
1932      1
1933      2
1934      1
1935      1
1936      2
1937      1
1938      2
1939      2
1940      4
1941      1
1942      1
1944      2
1945      4
1946      3
1947      3
1948      3
1949      2
1950      1
1951      3
1952      4
1953      4
1954      4
1955      2
1956      3
1957      2
       ... 
1988     28
1989     28
1990     24
1991     29
1992     29
1993     44
1994     50
1995     64
1996     86
1997    102
1998    119
1999    145
2000    145
2001    159
2002    164
2003    131
2004    149
2005    149
2006    144
2007     92
2008     97
2009     85
2010     54
2011     43
2012     92
2013    127
2014    157
2015    110
2016     31
nan       1
Name: id, Length: 90, dtype: int64

4 Extract the domain of each homepage.

In [7]:
# definisco una regex che vada a pulire i domini
def find_domain(string):
    string=re.sub("(.*)\//", '', string)
    string=re.sub("www\.", '', string)
    string=re.sub("\/.*", '', string)
    #string=re.sub('\.com.*|\.net.*|\.co\.uk*|\.org.*', '', string)
    return string

In [8]:
domains=[]
for index,row in df.iterrows():
    domains.append(find_domain(str(row["homepage"])))
df["domain"]=domains

In [9]:
#controllo se i NaN son rimasti lo stesso numero
df["homepage"].isnull().sum()==(df["domain"]=='nan').sum()

True

In [10]:
#creo la colonna "domain" facendo lo split della colonna "homepage" e prendendo la seconda rispetto al doppio slash, la prima rispetto allo slash e l'ultima divisione rispetto al punto 
#df["domain"]=df["homepage"].str.split(" ").str[0].str.split("//").str[1].str.split("/").str[0].str.split(".").str[-1]
df["domain"]

0                        avatarmovie.com
1                          disney.go.com
2                       sonypictures.com
3                 thedarkknightrises.com
4                      movies.disney.com
5                       sonypictures.com
6                          disney.go.com
7                             marvel.com
8             harrypotter.warnerbros.com
9       batmanvsupermandawnofjustice.com
10                          superman.com
11                               mgm.com
12                         disney.go.com
13                         disney.go.com
14                        manofsteel.com
15                                   nan
16                            marvel.com
17                         disney.go.com
18                      sonypictures.com
19                         thehobbit.com
20               theamazingspiderman.com
21                 robinhoodthemovie.com
22                         thehobbit.com
23                goldencompassmovie.com
24              

5 Extract a set of normalized tables. That is, each entry of a normalized table must contain exactly one value (not a list or a dictionary). There is no need to use SQL for this point.

In [11]:
#Creo tabella movie_id con i campi da non normalizzare
movie_id = df[['id', 'budget', 'domain', 'original_language', 'original_title', 'overview', 'popularity', 'release_date','year', 'revenue', 'runtime', 'status', 'tagline', 'title', 'vote_average', 'vote_count' ]]
movie_id = movie_id.rename(index=str, columns={'id': 'movie_id'})
movie_id = movie_id.set_index(['movie_id'])
movie_id.tail(5)

,budget,domain,original_language,original_title,overview,popularity,release_date,year,revenue,runtime,status,tagline,title,vote_average,vote_count
movie_id,,,,,,,,,,,,,,,
9367,220000,nan,es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,1992-09-04,1992,2040920,81.0,Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
72766,9000,nan,en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,2011-12-26,2011,0,85.0,Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
231617,0,hallmarkchannel.com,en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,2013-10-13,2013,0,120.0,Released,NaN,"Signed, Sealed, Delivered",7.0,6
126186,0,shanghaicalling.com,en,Shanghai Calling,When ambitious New York attorney Sam is sent t...,0.857008,2012-05-03,2012,0,98.0,Released,A New Yorker in Shanghai,Shanghai Calling,5.7,7
25975,0,nan,en,My Date with Drew,Ever since the second grade when he first saw ...,1.929883,2005-08-05,2005,0,90.0,Released,NaN,My Date with Drew,6.3,16


In [12]:
#Collego movie_id con i relativi generi
genere_id = pd.DataFrame()
for i in range(len(df['genres'])):
    unrolled_data = pd.read_json(df['genres'][i])
    temp_id = df['id'][i]
    unrolled_data['movie_id'] = temp_id
    genere_id = pd.concat([genere_id, unrolled_data])
genere_id = genere_id.rename(index=str, columns={'id': 'genere_id'})
genere = genere_id[['genere_id', 'name']]
genere_id = genere_id[['movie_id','genere_id']].set_index(['movie_id','genere_id'])
genere_id.tail()

Empty DataFrame
Columns: []
Index: [(231617, 35.0), (231617, 18.0), (231617, 10749.0), (231617, 10770.0), (25975, 99.0)]

In [13]:
# Creo la tablella di riferimento per i generi
genere = genere.set_index(['genere_id'])
genere = genere.drop_duplicates() 
genere.head()

,name
genere_id,
28.0,Action
12.0,Adventure
14.0,Fantasy
878.0,Science Fiction
80.0,Crime


In [14]:
# creo la tebella degli id delle varie keywords
keywords_ids = pd.DataFrame()
for i in range(len(df['keywords'])):
    unrolled_data = pd.read_json(df['keywords'][i])
    temp_id = df['id'][i]
    unrolled_data['movie_id'] = temp_id
    keywords_ids = pd.concat([keywords_ids, unrolled_data])
keywords_ids = keywords_ids.rename(index=str, columns={'id': 'keywords_id'})
keywords_link = keywords_ids[['keywords_id', 'name']]
keywords_ids = keywords_ids[['movie_id', 'keywords_id']].set_index(['movie_id', 'keywords_id'])
keywords_ids.head(10)

Empty DataFrame
Columns: []
Index: [(19995, 1463.0), (19995, 2964.0), (19995, 3386.0), (19995, 3388.0), (19995, 3679.0), (19995, 3801.0), (19995, 9685.0), (19995, 9840.0), (19995, 9882.0), (19995, 9951.0)]

In [15]:
#Creo la tabella che indica le keywords per ogni id
keywords_link = keywords_link.set_index(['keywords_id'])
keywords_link = keywords_link.drop_duplicates()
keywords_link.head()

,name
keywords_id,
1463.0,culture clash
2964.0,future
3386.0,space war
3388.0,space colony
3679.0,society


In [16]:
#Creo la tebella degli id delle compagnie di produzione
production_id = pd.DataFrame()
for i in range(len(df['production_companies'])):
    unrolled_data = pd.read_json(df['production_companies'][i])
    temp_id = df['id'][i]
    unrolled_data['movie_id'] = temp_id
    production_id = pd.concat([production_id, unrolled_data])
production_id = production_id.rename(index=str, columns={'id': 'company_id'})
production_link = production_id[['company_id', 'name']]
production_id = production_id[['movie_id', 'company_id']].set_index(['movie_id', 'company_id'])
production_id.head(10)

Empty DataFrame
Columns: []
Index: [(19995, 289.0), (19995, 306.0), (19995, 444.0), (19995, 574.0), (285, 2.0), (285, 130.0), (285, 19936.0), (206647, 5.0), (206647, 10761.0), (206647, 69434.0)]

In [16]:
#collego gli id ai nomi
#production_link = production_link.set_index(['company_id'])
production_link = production_link.drop_duplicates()
production_link.head()

,company_id,name
0.0,289.0,Ingenious Film Partners
1.0,306.0,Twentieth Century Fox Film Corporation
2.0,444.0,Dune Entertainment
3.0,574.0,Lightstorm Entertainment
0.0,2.0,Walt Disney Pictures


In [17]:
#creo le associazioni tra i i e le sigle(chiavi) dei paesi di produzione
countries_id = pd.DataFrame()
for i in range(len(df['production_countries'])):
    unrolled_data = pd.read_json(df['production_countries'][i])
    temp_id = df['id'][i]
    unrolled_data['movie_id'] = temp_id
    countries_id = pd.concat([countries_id, unrolled_data])
countries_id = countries_id.rename(index=str, columns={'iso_3166_1': 'country_id'})
countries_link = countries_id[['country_id', 'name']]
countries_id = countries_id[['movie_id', 'country_id']].set_index(['movie_id', 'country_id'])
countries_id.head()

Empty DataFrame
Columns: []
Index: [(19995, US), (19995, GB), (285, US), (206647, GB), (206647, US)]

In [18]:
# Creo la tabella delle associazioni
countries_link = countries_link.set_index(['country_id'])
countries_link = countries_link.drop_duplicates()
countries_link.head()

,name
country_id,
US,United States of America
GB,United Kingdom
JM,Jamaica
BS,Bahamas
DM,Dominica


In [19]:
#id dei i con le sigle dei linguaggi
languages_id = pd.DataFrame()
for i in range(len(df['spoken_languages'])):
    unrolled_data = pd.read_json(df['spoken_languages'][i])
    temp_id = df['id'][i]
    unrolled_data['movie_id'] = temp_id
    languages_id = pd.concat([languages_id, unrolled_data])
languages_id = languages_id.rename(index=str, columns={'iso_639_1': 'language_id'})
languages_link = languages_id[['language_id', 'name']]
languages_id = languages_id[['movie_id', 'language_id']].set_index(['movie_id', 'language_id'])
languages_id.head()

Empty DataFrame
Columns: []
Index: [(19995, en), (19995, es), (285, en), (206647, fr), (206647, en)]

In [20]:
# id dei linguaggi e il loro corrispettivo
languages_link = languages_link.set_index(['language_id'])
languages_link = languages_link.drop_duplicates()
languages_link.head()

,name
language_id,
en,English
es,Español
fr,Français
it,Italiano
de,Deutsch


In [21]:
#id dei credit 
cast = pd.DataFrame()
for i in range(len(df['cast'])):
    unrolled_data = pd.read_json(df['cast'][i])
    temp_id = df['movie_id'][i]
    unrolled_data['movie_id'] = temp_id
    cast = pd.concat([cast, unrolled_data])
cast = cast.rename(index=str, columns={'id': 'person_id'})
#cast_id = cast[['movie_id', 'credit_id']]
cast_id = cast[['movie_id', 'cast_id', 'person_id', 'credit_id', 'character','order']]
cast_id = cast_id.set_index(['credit_id'])
cast_id.head()

,movie_id,cast_id,person_id,character,order
credit_id,,,,,
5602a8a7c3a3685532001c9a,19995,242.0,65731.0,Jake Sully,0.0
52fe48009251416c750ac9cb,19995,3.0,8691.0,Neytiri,1.0
52fe48009251416c750aca39,19995,25.0,10205.0,Dr. Grace Augustine,2.0
52fe48009251416c750ac9cf,19995,4.0,32747.0,Col. Quaritch,3.0
52fe48009251416c750ac9d3,19995,5.0,17647.0,Trudy Chacon,4.0


In [22]:
#id crew
crew = pd.DataFrame()
for i in range(len(df['crew'])):
    unrolled_data = pd.read_json(df['crew'][i])
    temp_id = df['movie_id'][i]
    unrolled_data['movie_id'] = temp_id
    crew = pd.concat([crew, unrolled_data])
crew = crew.rename(index=str, columns={'id': 'person_id'})
#crew_id = crew[['movie_id', 'credit_id']]
crew_id = crew[['movie_id', 'credit_id', 'person_id', 'job', 'department']]
crew_id = crew_id.set_index(['credit_id'])
crew_id.head()

,movie_id,person_id,job,department
credit_id,,,,
52fe48009251416c750aca23,19995,1721.0,Editor,Editing
539c47ecc3a36810e3001f87,19995,496.0,Production Design,Art
54491c89c3a3680fb4001cf7,19995,900.0,Sound Designer,Sound
54491cb70e0a267480001bd0,19995,900.0,Supervising Sound Editor,Sound
539c4a4cc3a36810c9002101,19995,1262.0,Casting,Production


In [23]:
#unisco crew e cast
people_crew = crew[['person_id', 'gender', 'name']]
people_cast = cast[['person_id', 'gender', 'name']]
people = pd.concat([people_crew, people_cast])
people = people.sort_values('person_id').drop_duplicates().set_index('person_id')
people.head()

,gender,name
person_id,,
1.0,2.0,George Lucas
2.0,2.0,Mark Hamill
3.0,2.0,Harrison Ford
4.0,1.0,Carrie Fisher
5.0,2.0,Peter Cushing


6 For each movie, compute the gross margin (difference between revenue and budget)

In [60]:
df["gross_margin"]=df["revenue"]-df["budget"]
df[["movie_id","gross_margin"]].head(10)

,movie_id,gross_margin
0,19995,2550965087
1,285,661000000
2,206647,635674609
3,49026,834939099
4,49529,24139100
5,559,632871626
6,38757,331794936
7,99861,1125403694
8,767,683959197
9,209112,623260194


7 For each movie, compute the number of crew members

In [25]:
#trovo numero di membri della crew contando il numero di sottodizionari nel file JSON "crew"
df["no. crew members"]=df.crew.apply(lambda x: len(x))

8 For each movie, compute the number of directors

In [26]:
#trovo numero di directors controllando la lunghezza del vettore dir dove compaiono tutti i Director del film 
df['crew']=df.crew.apply(json.loads)
df["no. directors"]=df.crew.apply(lambda crew_members: len([dir for dir in crew_members if dir["job"]=="Director"]))

9 For each language, compute the number of movies where such language is spoken.

In [61]:
#no. movies for spoken_language
df["spoken_languages"]=df.spoken_languages.apply(json.loads)
df["spoken_languages_ids"]=df.spoken_languages.apply(lambda x: [lang['name'] for lang in x])
count_movies_by_lang={}
for lang in set(np.concatenate(df['spoken_languages_ids'])):
    count_movies_by_lang[lang]=list(np.concatenate(df['spoken_languages_ids'])).count(lang)
count_movies_by_lang
df_lang=pd.DataFrame.from_dict(count_movies_by_lang,orient='index')
df_lang=df_lang.rename(columns={df_lang.columns[0]:"No. Films"})

df_lang.head(10)

,No. Films
,54
اردو,15
Nederlands,10
తెలుగు,1
No Language,2
Italiano,188
isiZulu,4
Afrikaans,7
Latin,52
Türkçe,18


10 For each company and each decade, compute the overall revenue

In [59]:
#revenues for production_company
df["production_companies"]=df.production_companies.apply(json.loads)
df["production_companies_ids"]=df.production_companies.apply(lambda x: [comp['name'] for comp in x])
count_companies_revenue={}
for index,row in df.iterrows():
        for comp in row["production_companies_ids"]:
            try:
                cont=count_companies_revenue[comp]
            except:
                cont=0
            count_companies_revenue[comp]=cont+row['revenue']
count_companies_revenue

df_revenue_comp=pd.DataFrame.from_dict(count_companies_revenue,orient='index')
df_revenue_comp=df_revenue_comp.rename(columns={df_revenue_comp.columns[0]:"Revenue($)"})
df_revenue_comp.head(10)

,Revenue($)
Ingenious Film Partners,7907230983
Twentieth Century Fox Film Corporation,39357151309
Dune Entertainment,13797504190
Lightstorm Entertainment,5621881784
Walt Disney Pictures,28683256048
Jerry Bruckheimer Films,7213427769
Second Mate Productions,2026659812
Columbia Pictures,28599634775
Danjaq,1727236985
B24,880674609


In [57]:
#revenues for companies and decades
df["decade"]=df.year.str[:3]+"0"
count_companies_revenue_decades={}
for index,row in df.iterrows():
        for comp in row["production_companies_ids"]:
            try:
                cont=count_companies_revenue_decades[comp][row['decade']]
            except:
                try:
                    count_companies_revenue_decades[comp][row['decade']]={}
                    cont=0
                except:
                    count_companies_revenue_decades[comp]={}
                    count_companies_revenue_decades[comp][row['decade']]={}
                    count_companies_revenue_decades[comp][row['decade']]
                    cont=0
            count_companies_revenue_decades[comp][row['decade']]=cont+row['revenue']
count_companies_revenue_decades

df_revenue_count_comp=pd.DataFrame.from_dict(count_companies_revenue_decades["Warner Bros."],orient='index')
df_revenue_count_comp=df_revenue_count_comp.rename(columns={df_revenue_count_comp.columns[0]:"Revenue($)"})
df_revenue_count_comp.index.names=["Warner Bros"]
df_revenue_count_comp.head(10)

,Revenue($)
Warner Bros,
2010,14574154994
2000,22897419289
1990,8665268956
1980,1827894714
1970,1060259690
1960,84070731
1950,31200000
1930,5017000
1940,10462500


11 For each decade, compute the company with maximum revenue

In [55]:
max_companies_for_decade={}
max_revenue_for_decade={}
for decade in set(df[df['decade']!='nan0']["decade"]):
    max_companies_for_decade[decade]="NaN"
    max_revenue_for_decade[decade]=0
    for company in count_companies_revenue_decades.keys():
        try:
            if count_companies_revenue_decades[company][decade]>max_revenue_for_decade[decade]:
                max_companies_for_decade[decade]=company
                max_revenue_for_decade[decade]=count_companies_revenue_decades[company][decade]
        except:
            pass
max_companies_for_decade

df_revenue_max=pd.DataFrame.from_dict(max_companies_for_decade,orient='index')
df_revenue_max=df_revenue_max.rename(columns={df_revenue_max.columns[0]:"Company"})
df_revenue_max.head(10)

,Company
2000,Warner Bros.
1920,Metro-Goldwyn-Mayer (MGM)
1970,United Artists
1990,Paramount Pictures
1960,Eon Productions
1930,Metro-Goldwyn-Mayer (MGM)
1910,Triangle Film Corporation
1940,Walt Disney Productions
1950,Metro-Goldwyn-Mayer (MGM)
2010,Universal Pictures


12 In each year, how many movies have revenue smaller than the budget?

In [54]:
fails4year={}
for index,row in df.iterrows():
    if row["revenue"]<row["budget"]:
        try:
            fails4year[row['year']]+=1
        except:
            fails4year[row['year']]=1
fails4year

fails_per_year=pd.DataFrame.from_dict(fails4year,orient='index')
fails_per_year=fails_per_year.rename(columns={fails_per_year.columns[0]:"Failures"})
fails_per_year.sort_values('Failures',ascending=False).head(10)

,Failures
2009,78
2002,71
2011,70
2015,67
2010,65
2008,64
2006,63
2013,62
2004,61
2014,59


### Punti da due persone

1 Distribute the revenue according to the order of appearance in a movie. Assume that the i-th actor contributes twice as much as the (i+1)-th actor to the revenue.

In [23]:
df["cast"]=df.cast.apply(json.loads)
df['actors']=df.cast.apply(lambda x: [act['name'] for act in x])


In [24]:
revenue={}
for index,row in df.iterrows():
    temp={}
    for i in range(len(row["actors"])):
        temp[row["actors"][i]]=(row["revenue"]/2**(i+1))
    revenue[row["original_title"]]=temp 

In [26]:
chosen_movie='Interstellar'
df_revenue=pd.DataFrame.from_dict(revenue[chosen_movie],orient='index')
df_revenue=df_revenue.rename(columns={df_revenue.columns[0]:"Contribution($)"})
df_revenue.index.names=[chosen_movie]
pd.options.display.float_format = '{:,.2f}'.format
df_revenue.head(10)


,Contribution($)
Interstellar,
Matthew McConaughey,"337,560,008.50"
Jessica Chastain,"168,780,004.25"
Anne Hathaway,"84,390,002.12"
Michael Caine,"42,195,001.06"
Casey Affleck,"21,097,500.53"
Mackenzie Foy,"10,548,750.27"
Timothée Chalamet,"5,274,375.13"
Bill Irwin,"2,637,187.57"
Matt Damon,"1,318,593.78"


2 For each actor find the total revenue attributed to him/her.

In [48]:
#overall revenue for actor
df['actors']=df.cast.apply(lambda x: [act['name'] for act in x])
revenue4actor={}
for index,row in df.iterrows():
    n=len(row['actors'])
    for i in range(n):
        try:
            revenue4actor[row['actors'][i]]=int(revenue4actor[row['actors'][i]]+row['revenue']/2**(i+1))
        except:
            revenue4actor[row['actors'][i]]=int(row['revenue']/2**(i+1))

In [53]:
df_revenue_actor=pd.DataFrame.from_dict(dict(revenue4actor),orient='index')
df_revenue_actor=df_revenue_actor.rename(columns={df_revenue_actor.columns[0]:"Revenue"})
df_revenue_actor.sort_values('Revenue',ascending=False).head()

,Revenue
Tom Cruise,3976017164
Tom Hanks,3799897888
Robert Downey Jr.,3631679400
Johnny Depp,3511503057
Will Smith,3320046077


3 Find the actor that is responsible for the most overall revenue.

In [37]:
revenue4actor=sorted(revenue4actor.items(), key=operator.itemgetter(1),reverse=True)


In [38]:
revenue4actor[0]

('Tom Cruise', 3976017164)